### 最短路径问题

<img src="images/shortestpath.png">

In [1]:
# 邻接表
graph_dict = {0:{1:6,2:3},
              1:{0:6,2:8,3:7,6:4},
              2:{0:3,1:8,4:2,5:1,7:10},
              3:{1:7},
              4:{2:2,6:1},
              5:{2:1,7:8},
              6:{1:4,4:1},
              7:{2:10,5:8}}

In [2]:
from heapq import *

# Dijkstra算法，不断更新最近节点的相邻节点
def dijkstra(graph_dict, start, end):
    costs = {} # 记录每个节点的临时最短路径
    processed = set() # 记录已经确定最短路径的节点
    heap = []
    costs[start] = (0, None) # 将起点信息放入costs中，格式为（距离，前节点）
    heappush(heap, (0, start, None)) # 将起点信息放入最小堆中，格式为（距离，当前节点，前节点）
    
    while heap:
        # 从堆顶取出当前最短路径
        cost, cur_node, pre_node = heappop(heap)
        while cur_node in processed:
            if not heap: return None, []
            cost, cur_node, pre_node = heappop(heap)
        # 如果当前节点为终点，则返回最短距离和最短路径
        if cur_node == end:
            path = []
            x = cur_node
            while x is not None:
                path.insert(0, x)
                _, x = costs[x]
            return cost, path
        # 将当前节点标记为已经确定最短路径的节点
        processed.add(cur_node)
        # 对当前节点的所有相邻节点，更新起点到相邻节点的最短路
        for adj_node in graph_dict[cur_node]:
            if adj_node in processed: continue # 跳过已确定最短路径的节点
            new_cost = cost + graph_dict[cur_node][adj_node] # 新距离 = 已确定的当前节点距离 + 邻边长度
            if adj_node in costs and new_cost >= costs[adj_node][0]: continue # 原有路径更短则无需更新
            costs[adj_node] = new_cost, cur_node
            heappush(heap, (new_cost, adj_node, cur_node))
    
    return None, []

In [3]:
dijkstra(graph_dict, 1, 5)

(8, [1, 6, 4, 2, 5])

In [4]:
dijkstra(graph_dict, 6, 0)

(6, [6, 4, 2, 0])

In [5]:
dijkstra(graph_dict, 0, 0)

(0, [0])

In [6]:
# Bellman算法，允许存在无环负权边，每轮更新所有边，共N-1轮次
def bellman(graph_dict, start, end):
    costs = {}
    costs[start] = (0, None)
    
    # 循环N-1轮次
    for time in range(len(graph_dict) - 1):
        # 遍历已经有计算过距离的所有节点
        for cur_node in costs.copy():
            # 遍历当前节点的相邻节点
            for adj_node in graph_dict[cur_node]:
                new_cost = costs[cur_node][0] + graph_dict[cur_node][adj_node]
                if adj_node in costs and new_cost >= costs[adj_node][0]: continue
                costs[adj_node] = (new_cost, cur_node)
    
    # 回溯最短路径
    path = []
    x = end
    while x is not None:
        path.insert(0, x)
        _, x = costs[x]
    
    return costs[end][0], path

In [7]:
bellman(graph_dict, 1, 5)

(8, [1, 6, 4, 2, 5])

In [8]:
bellman(graph_dict, 6, 0)

(6, [6, 4, 2, 0])

In [9]:
bellman(graph_dict, 0, 0)

(0, [0])

### LeetCode 743. 网络延迟时间

有 N 个网络节点，标记为 1 到 N。

给定一个列表 times，表示信号经过有向边的传递时间。 times[i] = (u, v, w)，其中 u 是源节点，v 是目标节点， w 是一个信号从源节点传递到目标节点的时间。

现在，我们向当前的节点 K 发送了一个信号。需要多久才能使所有节点都收到信号？如果不能使所有节点收到信号，返回 -1。

注意:

N 的范围在 [1, 100] 之间。
K 的范围在 [1, N] 之间。
times 的长度在 [1, 6000] 之间。
所有的边 times[i] = (u, v, w) 都有 1 <= u, v <= N 且 0 <= w <= 100。

<img src="images/directweightedgraph.png">

In [10]:
times = [(0,1,20), (0,2,15),
         (1,0,2), (1,4,10), (1,5,30),
         (2,1,4), (2,5,10),
         (4,3,15),
         (5,3,4), (5,4,10)
        ]

In [11]:
from heapq import *

# 使用Dijkstra算法，不断更新（离起点）最近节点的相邻节点
def network_delay_time(times, N, K):
    
    # 将边权重列表转换为邻接表
    def get_graph(times, N):
        graph = {}
        for i in range(N):
            graph[i] = {}
        for begin, end, weight in times:
            if begin not in graph: graph[begin] = {}
            if end not in graph: graph[end] = {}
            graph[begin][end] = weight
        return graph
    
    graph = get_graph(times, N)
    processed = set() # 记录已经确定最短路径的节点
    costs = {} # 记录每个节点的临时最短路径
    heap = [] # 最小堆，储存每个节点的临时最短路径
    
    costs[K] = 0
    heappush(heap, (0, K))
    
    while heap:
        cost, cur_node = heappop(heap) # 从堆顶取出当前最短路径
        while cur_node in processed:
            if not heap: break
            cost, cur_node = heappop(heap)
        if cur_node in processed: break
        # 对当前节点的所有相邻节点，更新起点到相邻节点的最短路
        for adj_node in graph[cur_node]:
            if adj_node in processed: continue # 跳过已确定最短路径的节点
            new_cost = cost + graph[cur_node][adj_node] # 新距离 = 已确定的当前节点距离 + 邻边长度
            if adj_node in costs and new_cost >= costs[adj_node]: continue # 原有路径更短则无需更新
            costs[adj_node] = new_cost
            heappush(heap, (new_cost, adj_node))
        processed.add(cur_node) # 将当前节点标记为已经确定最短路径的节点
        
    if len(processed) < N: return -1
    
    max_cost = max([costs[i] for i in processed]) # 最大的最短路径距离，即为所有节点收到信号的时间
    
    return max_cost

In [12]:
network_delay_time(times, 6, 0)

29

In [13]:
network_delay_time(times, 6, 1)

27

In [14]:
network_delay_time(times, 6, 3)

-1

In [15]:
network_delay_time(times, 6, 5)

-1

### LeetCode 207. 课程表

现在你总共有 n 门课需要选，记为 0 到 n-1。

在选修某些课程之前需要一些先修课程。 例如，想要学习课程 0 ，你需要先完成课程 1 ，我们用一个匹配来表示他们: [0,1]

给定课程总量以及它们的先决条件，判断是否可能完成所有课程的学习？

示例 1:

输入: 2, [[1,0]] 
输出: true

解释: 总共有 2 门课程。学习课程 1 之前，你需要完成课程 0。所以这是可能的。
示例 2:

输入: 2, [[1,0],[0,1]]
输出: false

解释: 总共有 2 门课程。学习课程 1 之前，你需要先完成​课程 0；并且学习课程 0 之前，你还应先完成课程 1。这是不可能的。

说明:

输入的先决条件是由边缘列表表示的图形，而不是邻接矩阵。详情请参见图的表示法。
你可以假定输入的先决条件中没有重复的边。

提示:

这个问题相当于查找一个循环是否存在于有向图中。如果存在循环，则不存在拓扑排序，因此不可能选取所有课程进行学习。
通过 DFS 进行拓扑排序 - 一个关于Coursera的精彩视频教程（21分钟），介绍拓扑排序的基本概念。
拓扑排序也可以通过 BFS 完成。

In [16]:
# 深度优先遍历
def can_finish(num_courses, prerequisites):
    graph = {} # 邻接表
    visited = {} # 访问状态 0-未访问 1-已访问过 2-正在遍历中
    
    # 生成邻接表
    for i in range(num_courses):
        graph[i] = set()
        visited[i] = 0
    
    for end, begin in prerequisites:
        graph[begin].add(end)
    
    # 深度优先遍历，判断是否有环
    def dfs(node):
        if visited[node] == 2: return True # 遇到正在遍历的节点，表示存在环
        if visited[node] != 0: return False # 遇到已经访问过的节点，结束访问
        
        visited[node] = 2 # 标记为正在遍历
        for successor in graph[node]:
            if dfs(successor): # 遇到环
                return True
        
        visited[node] = 1 # 标记为已访问
        return False
    
    for node in graph:
        if dfs(node): return False
    
    return True

In [17]:
can_finish(2, [[1,0], [0,1]])

False

In [18]:
can_finish(4, [[1,0], [2,1], [3,1]])

True

In [19]:
can_finish(3, [[0,2], [1,0]])

True

In [20]:
can_finish(4, [[1,0], [2,1], [3,2], [0,3]])

False

In [21]:
can_finish(4, [[1,0], [2,1], [0,2]])

False

In [22]:
can_finish(4, [[1,0], [3,2]])

True

In [23]:
can_finish(1, [])

True

### LeetCode 210. 课程表 II

现在你总共有 n 门课需要选，记为 0 到 n-1。

在选修某些课程之前需要一些先修课程。 例如，想要学习课程 0 ，你需要先完成课程 1 ，我们用一个匹配来表示他们: [0,1]

给定课程总量以及它们的先决条件，返回你为了学完所有课程所安排的学习顺序。

可能会有多个正确的顺序，你只要返回一种就可以了。如果不可能完成所有课程，返回一个空数组。

示例 1:

输入: 2, [[1,0]] 
输出: [0,1]

解释: 总共有 2 门课程。要学习课程 1，你需要先完成课程 0。因此，正确的课程顺序为 [0,1] 。
示例 2:

输入: 4, [[1,0],[2,0],[3,1],[3,2]]
输出: [0,1,2,3] or [0,2,1,3]

解释: 总共有 4 门课程。要学习课程 3，你应该先完成课程 1 和课程 2。并且课程 1 和课程 2 都应该排在课程 0 之后。
     因此，一个正确的课程顺序是 [0,1,2,3] 。另一个正确的排序是 [0,2,1,3] 。
说明:

输入的先决条件是由边缘列表表示的图形，而不是邻接矩阵。详情请参见图的表示法。
你可以假定输入的先决条件中没有重复的边。

提示:

这个问题相当于查找一个循环是否存在于有向图中。如果存在循环，则不存在拓扑排序，因此不可能选取所有课程进行学习。
通过 DFS 进行拓扑排序 - 一个关于Coursera的精彩视频教程（21分钟），介绍拓扑排序的基本概念。
拓扑排序也可以通过 BFS 完成。

In [24]:
# 拓扑排序，每一次都输出入度为 0 的结点，并移除它、修改它指向的结点的入度，依次得到的结点序列就是拓扑排序的结点序列
def find_order(num_courses, prerequisites):
    in_degrees = {} # 入度表
    graph = {} # 邻接表
    
    # 生成入度表和邻接表
    for i in range(num_courses):
        in_degrees[i] = 0    
        graph[i] = set()
    
    for end, begin in prerequisites:
        in_degrees[end] += 1   
        graph[begin].add(end)
        
    queue = [] # 存放入度为0的节点的队列
    for node in in_degrees:
        if in_degrees[node] == 0:
            queue.append(node)
    
    res = [] # 访问序列
    while queue:
        node = queue.pop(0)
        res.append(node)
        
        # 遍历后继节点，并使其入度减1，如果入度为0，则加入队列
        for successor in graph[node]:
            in_degrees[successor] -= 1
            if in_degrees[successor] == 0:
                queue.append(successor)
    
    # 结果序列未包含所有节点，说明有环
    if len(res) != num_courses:
        return []
    
    return res

In [25]:
find_order(2, [[1,0]])

[0, 1]

In [26]:
find_order(4, [[1,0],[2,0],[3,1],[3,2]])

[0, 1, 2, 3]

In [27]:
find_order(4, [[1,0], [2,1], [3,1]])

[0, 1, 2, 3]

In [28]:
find_order(3, [[0,2], [1,0]])

[2, 0, 1]

In [29]:
find_order(4, [[1,0], [2,1], [3,2], [0,3]])

[]

In [30]:
find_order(4, [[1,0], [2,1], [0,2]])

[]

In [31]:
find_order(4, [[1,0], [3,2]])

[0, 2, 1, 3]

In [32]:
find_order(1, [])

[0]

### LeetCode 630. 课程表 III

这里有 n 门不同的在线课程，他们按从 1 到 n 编号。每一门课程有一定的持续上课时间（课程时间）t 以及关闭时间第 d 天。一门课要持续学习 t 天直到第 d 天时要完成，你将会从第 1 天开始。

给出 n 个在线课程用 (t, d) 对表示。你的任务是找出最多可以修几门课。

 

示例：

输入: [[100, 200], [200, 1300], [1000, 1250], [2000, 3200]]
输出: 3

解释: 
这里一共有 4 门课程, 但是你最多可以修 3 门:
首先, 修第一门课时, 它要耗费 100 天，你会在第 100 天完成, 在第 101 天准备下门课。
第二, 修第三门课时, 它会耗费 1000 天，所以你将在第 1100 天的时候完成它, 以及在第 1101 天开始准备下门课程。
第三, 修第二门课时, 它会耗时 200 天，所以你将会在第 1300 天时完成它。
第四门课现在不能修，因为你将会在第 3300 天完成它，这已经超出了关闭日期。
 

提示:

整数 1 <= d, t, n <= 10,000 。
你不能同时修两门课程。

In [33]:
# 深度优先搜索
def schedule_course(courses):
    
    # dfs表示当前时间能学完的最多剩余课程
    def dfs(time, left_courses):
        # 从剩余课程中找出所有符合要求的课程
        max_count = 0
        for course in left_courses:
            cost, deadline = courses[course]
            if time <= deadline - cost:
                next_courses = left_courses.copy()
                next_courses.remove(course)
                max_count = max(max_count, 1 + dfs(time + cost, next_courses))
        return max_count
    
    res = dfs(0, [i for i in range(len(courses))])
    
    return res

In [34]:
schedule_course([[100, 200], [200, 1300], [1000, 1250], [2000, 3200]])

3

In [35]:
schedule_course([[100, 200], [100, 200]])

2

In [36]:
schedule_course([[100, 200], [101, 200]])

1

In [37]:
schedule_course([])

0

In [38]:
# 动态规划，贪心算法
def schedule_course2(courses):
    if not courses: return 0
    
    num_courses = len(courses)
    
    # 按照结束时间排序，若结束时间相同，则用时短的在前
    sorted_courses = sorted(courses, key=lambda x:(x[1], x[0]))
    
    # dp[i]表示学完i门课需要的最短时间
    dp = [float('inf')] * (num_courses + 1)
    dp[0] = 0
    
    res = 0 # 记录最多可以学的课程数
    for i in range(num_courses):
        t, d = sorted_courses[i]
        for j in reversed(range(i + 1)): # 从后往前更新
            new_dp = t + dp[j]
            # 如果这节课可以在结束时间上完，并且时长小于之前的（贪心），则优化
            if new_dp <= d and new_dp < dp[j + 1]:
                dp[j + 1] = new_dp
                res = max(res, j + 1)
    
    return res

In [39]:
schedule_course2([[100, 200], [200, 1300], [1000, 1250], [2000, 3200]])

3

In [40]:
schedule_course2([[100, 200], [100, 200]])

2

In [41]:
schedule_course2([[100, 200], [101, 200]])

1

In [42]:
schedule_course2([])

0

### LeetCode 329. 矩阵中的最长递增路径

给定一个整数矩阵，找出最长递增路径的长度。

对于每个单元格，你可以往上，下，左，右四个方向移动。 你不能在对角线方向上移动或移动到边界外（即不允许环绕）。

示例 1:

输入: nums = 
[
  [9,9,4],
  [6,6,8],
  [2,1,1]
] 
输出: 4 

解释: 最长递增路径为 [1, 2, 6, 9]。

示例 2:

输入: nums = 
[
  [3,4,5],
  [3,2,6],
  [2,2,1]
] 
输出: 4 

解释: 最长递增路径是 [3, 4, 5, 6]。注意不允许在对角线方向上移动。

In [43]:
# 记忆化深度优先搜索
def longest_increasing_path(matrix):
    if not matrix or not matrix[0]: return 0
    
    direct = [(-1, 0), (1, 0), (0, -1), (0, 1)]
    width = len(matrix[0])
    height = len(matrix)
    cache = [[0] * width for i in range(height)]
    
    def dfs(x, y):
        if cache[y][x] > 0: return cache[y][x]
        max_dis = 0
        for ix, iy in direct:
            nx, ny = x + ix, y + iy
            if nx < 0 or nx >= width or ny < 0 or ny >= height: continue
            if matrix[ny][nx] > matrix[y][x]:
                max_dis = max(max_dis, dfs(nx, ny))
        cache[y][x] = 1 + max_dis
        return cache[y][x]
    
    res = 0
    for y in range(height):
        for x in range(width):
            res = max(res, dfs(x, y))
    
    return res

In [44]:
matrix = [[9,9,4], [6,6,8], [2,1,1]]
longest_increasing_path(matrix)

4

In [45]:
matrix = [[3,4,5], [3,2,6], [2,2,1]]
longest_increasing_path(matrix)

4

In [46]:
# 拓扑排序，每一次都输出出度为 0 的结点（叶子），并移除它、修改指向它的结点的出度，统计其遍历深度
def longest_increasing_path2(matrix):
    if not matrix or not matrix[0]: return 0
    
    direct = [(-1, 0), (1, 0), (0, -1), (0, 1)]
    width = len(matrix[0])
    height = len(matrix)    
    out_degrees = [[0] * width for i in range(height)] # 出度表
    queue = [] # 出度为0的节点队列
    
    # 计算所有节点的出度
    for y in range(height):
        for x in range(width):
            for ix, iy in direct:
                nx, ny = x + ix, y + iy
                if nx >= 0 and nx < width and ny >= 0 and ny < height and matrix[ny][nx] > matrix[y][x]:
                    out_degrees[y][x] += 1
    
    # 将出度为0的节点加入队列
    for y in range(height):
        for x in range(width):
            if out_degrees[y][x] == 0:
                queue.append((x, y))
    
    height = 0 # 遍历深度
    while queue:
        height += 1
        new_queue = [] # 新队列
        # 遍历队列中所有节点的前驱节点
        for x, y in queue:
            for ix, iy in direct:
                px, py = x + ix, y + iy
                if px >= 0 and px < width and py >= 0 and py < height and matrix[py][px] < matrix[y][x]:
                    out_degrees[py][px] -= 1
                    if out_degrees[py][px] == 0:
                        new_queue.append((px, py)) # 如果前驱节点的出度变为0，则加入新队列
        queue = new_queue
    
    return height

In [47]:
matrix = [[9,9,4], [6,6,8], [2,1,1]]
longest_increasing_path2(matrix)

2

In [48]:
matrix = [[3,4,5], [3,2,6], [2,2,1]]
longest_increasing_path2(matrix)

4